In [1]:
import tensorflow as tf 

In [2]:
tf.__version__

'2.3.0'

In [3]:
(train_images, train_labels), _ = tf.keras.datasets.mnist.load_data()

In [4]:
train_images.shape

(60000, 28, 28)

In [5]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [6]:
train_images = tf.expand_dims(train_images, -1)

In [7]:
train_images.shape

TensorShape([60000, 28, 28, 1])

In [8]:
train_images = tf.cast(train_images/255, tf.float32)

In [9]:
train_labels = tf.cast(train_labels, tf.int64)

In [10]:
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

In [11]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [12]:
dataset = dataset.shuffle(10000).batch(32)

In [13]:
dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3, 3], activation='relu', input_shape=(None, None, 1)),
    tf.keras.layers.Conv2D(32, [3, 3], activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10)
])

In [15]:
optimizer = tf.keras.optimizers.Adam()

In [16]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [17]:
features, labels = next(iter(dataset))

In [18]:
features.shape

TensorShape([32, 28, 28, 1])

In [19]:
labels.shape

TensorShape([32])

In [20]:
predictions = model(features)

In [21]:
predictions.shape

TensorShape([32, 10])

In [22]:
tf.argmax(predictions, axis=1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([9, 0, 6, 0, 0, 0, 0, 0, 0, 6, 6, 9, 0, 9, 0, 0, 9, 0, 6, 0, 9, 0,
       0, 0, 6, 0, 9, 6, 6, 0, 9, 9], dtype=int64)>

In [23]:
labels

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([3, 3, 3, 6, 9, 2, 8, 4, 6, 7, 9, 2, 1, 7, 8, 9, 3, 4, 2, 1, 7, 2,
       3, 6, 9, 4, 5, 2, 4, 9, 7, 0], dtype=int64)>

In [24]:
def cal_loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)

In [25]:
def train_step(model, optimizer, images, labels):
    with tf.GradientTape() as tape:
        loss_step = cal_loss(model, images, labels)
    grads = tape.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [26]:
def train(epochs, model, optimizer, dataset):
    for epoch in range(epochs):
        for (batch_index, (images, labels)) in enumerate(dataset):
            train_step(model, optimizer, images, labels)
        print('Epoch{} is finshed'.format(epoch))

In [ ]:
train(10, model, optimizer, dataset)